In [17]:
import sys
import os
import json
import re
import ast

In [2]:
from dotenv import load_dotenv
load_dotenv()


True

In [3]:
from IPython.display import Markdown
display(Markdown('*some markdown* test'))

*some markdown* test

In [4]:
# @title google api key
# os.environ['GOOGLE_API_KEY'] = <api key>

In [5]:
from google import generativeai as genai
genai.configure()

model = genai.GenerativeModel('gemini-pro')


/home/sudesh/git/Automated interview/.py3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
# we need temprature as 1 to get consistent results
temp_0_config = genai.GenerationConfig( **{  
    # "candidate_count": int | None = None
    # "stop_sequences": Iterable[str] | None = None
    # "max_output_tokens": int | None = None
    "temperature": 0
    # "top_p": float | None = None
    # "top_k": int | None = None
    })
# model_response = model.generate_content("write a poem about machnical lemonade",generation_config=temp_0_config).text
# display(Markdown(model_response))

**Mechanical Lemonade**

In a realm of gears and steam,
Where innovation's dreams gleam,
A marvel arose, a wondrous sight,
Mechanical Lemonade, a pure delight.

Its pistons churned, a rhythmic beat,
As lemons danced, their essence sweet,
Through tubes and valves, a liquid gold,
A thirst-quenching elixir, a story to be told.

With every sip, a burst of zest,
A symphony of flavors, truly blessed,
Its tangy sweetness, a refreshing treat,
A taste of summer, a moment complete.

Its gears and cogs, a mesmerizing dance,
A testament to human brilliance,
A machine that brought joy to all,
Mechanical Lemonade, standing tall.

In bustling streets and crowded fairs,
It quenched the thirst, banished all cares,
A symbol of progress, a marvel to behold,
Mechanical Lemonade, a story to be told.

So raise a glass to this wondrous creation,
A testament to innovation's liberation,
Mechanical Lemonade, a timeless delight,
A taste of summer, a pure and bright.

In [7]:
def judge_answer(q,a,c):
  response = model.generate_content(f"""
you are a teaching assistent ai who will help students correct their answers.

for a given question, criteria and student answer tell the student what they did correct and where they have to improve their understanding of the subject.

input:
```
question: {q}
criteria: {c}
student answer: {a}

output format
```
Correct points:
1.
2.
...
Points to improve:
1.
2.
...

summery:
Overall,...
```

end your summary with words of Encouragement like good job! or keep up the effort etc.
do not repeate the inputs in your response

""",generation_config=temp_0_config)
  return response.text


FreeText can use question content to automatically establish grading criteria, or it can use the assessment criteria to improve the text of the question.
The latter process works by asking the AI to serve as
a student and answer a question while oblivious to the
instructor’s grading criteria. Then, the answer is automatically evaluated by a separate instantiation of the
LLM — this time, against the instructor criteria. The
assessment model determines if the student has been
unfairly penalized due to omission of requirements (or
a lack of clarity) in the original question text. If so, the
question is updated to better encompass the requirements of the grading criteria.

In [32]:
# @title Question evaluation
question = "What is the Rosetta Stone?" # @param {type:"string"}
criteria = "Mention why the Ptolemaic dynasty created the Rosetta Stone" # @param {type:"string"}

def refine_question_wrt_criteria(question,criteria):
  response = model.generate_content(f"""
  You are a model student with best grades.
  Answer the following question. try to be consise and presise.
  question:{question}

  do not repeate the input in your response
  """
  ,generation_config=temp_0_config)
  # display(Markdown(response.text))
  model_answer = response.text
  judjement = judge_answer(question,model_answer,criteria)
  # display(Markdown(judjement))


  #  The assessment model determines if the student has been unfairly penalized due to omission of requirements (or a lack of clarity) in the original question text

  response = model.generate_content(f'''
  You are a AI teaching assistent.
  for a given question, answer, evaluation criteria and evaluation.
  determine if the student has been unfairly penalized due to omission of requirements (or a lack of clarity) in the original question text.

  question: {question}
  criteria: {criteria}
  answer: {model_answer}
  evaluation: {judjement}

  output format
  ```
  student is being fairly/unfairly judged.

  reason:
  ```
  You need to strictly follow the output format.
  '''
  ,generation_config=temp_0_config)
  display(Markdown(response.text))
  # is_fair = re.search(r"your question is (fair|unfair)\.",response.text)
  is_fair = re.search(r"student is being (fairly|unfairly) judged\.",response.text)
  if is_fair is not None and is_fair[1].lower() == "unfairly":
      response = model.generate_content(f'''
      you are a teaching assistent AI.
      a question is evaluated based on the grading criteria.
      students are getting unfairly penalized due to omission of requirements (or a lack of clarity) in the original question text.
      given the question and grading criteria, suggest a list of alternative questions such that they better encompass the requirements of the grading criteria.

      strictly do not explicitly add grading criteria into the question.

      question: {question}
      grading criteria: {criteria}

      output json format
      {{"alternative questions": ["question 1","question 2",...]}}
      ''',generation_config=temp_0_config)
      # display(Markdown(response.text))
      print(response.text)

      return ast.literal_eval(response.text)
  else:
      return {
        "alternative questions": []
      }

print(refine_question_wrt_criteria(question,criteria))



student is being unfairly judged.

reason:
The original question does not mention anything about the Ptolemaic dynasty or why they created the Rosetta Stone. Therefore, the student cannot be penalized for not including this information in their answer.

{
  "alternative questions": [
    "Explain the purpose of the Rosetta Stone and its significance in understanding the Ptolemaic dynasty.",
    "Describe the historical context surrounding the creation of the Rosetta Stone, including the role of the Ptolemaic dynasty.",
    "Discuss the reasons why the Ptolemaic dynasty commissioned the creation of the Rosetta Stone and its impact on our understanding of ancient Egypt."
  ]
}
{'alternative questions': ['Explain the purpose of the Rosetta Stone and its significance in understanding the Ptolemaic dynasty.', 'Describe the historical context surrounding the creation of the Rosetta Stone, including the role of the Ptolemaic dynasty.', 'Discuss the reasons why the Ptolemaic dynasty commissioned the creation of the Rosetta Stone and its impact on our understanding of ancient Egypt.']}


In [11]:
# @title Student answer evaluation
question = "What are the components and structure of a molecule of DNA?" # @param {type:"string"}
criteria = "" # @param {type:"string"}
student_answer = "DNA is a complex molecule and it is shaped like a double helix ladder, where the rungs are base pairs ATGC and the scaffold is sugars and phosphates. The base pairs bind (A with G) and (C with T) using hydrogen bonds, which can be separated when the DNA is being read or duplicated" # @param {type:"string"}


display(Markdown(judge_answer(question,student_answer,criteria)))

**Correct points:**

1. You correctly described the double helix structure of DNA.
2. You correctly named the nitrogenous bases (ATGC) that form the rungs of the DNA ladder.

**Points to improve:**

1. You stated that the scaffold of DNA is made of sugars and phosphates. While this is partially correct, you should also mention that the scaffold consists of alternating units of deoxyribose sugar and phosphate groups.
2. You mentioned that the base pairs bind using hydrogen bonds, but you did not specify the specific pairing rules. You should state that adenine (A) always pairs with thymine (T), and cytosine (C) always pairs with guanine (G).

**Summary:**

Overall, your answer demonstrates a basic understanding of the structure of DNA. However, you should revise the details on the composition of the scaffold and the specific base pairing rules. Keep up the effort!